<!--
Credit to "prashant111" on Kaggle:
https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning
-->

# Predicting financial fraud

This is a notebook demonstrating how to analyze a dataset and make an XGBoost model using Bayesian Hyperparameter tuning to predict fraud with a high level of accuracy.


## Contents

1. The data

2. What is the XGBoost model?

3. Parameters of the XGBoost model

4. Bayesian Hyperparameter tuning

5. Why I use XGBoost and Bayesian Hyperparameter tuning

6. Code

# The data

The data is a large (493.53 MB) .csv file from kaggle.com ([link](https://www.kaggle.com/code/arjunjoshua/predicting-fraud-in-financial-payment-services/input)).


## Description

Paysim synthetic dataset of mobile money transactions. Each step represents an hour of simulation. This dataset is scaled down 1/4 of the original dataset which is presented in the paper "PaySim: A financial mobile money simulator for fraud detection".

## Shape

rows: 6,362,620

columns: 11

## Column names

### step
Maps a unit of time in the real world. In this case 1 step is 1 hour of time.

### type
Transaction type, either ```"CASH-IN"```, ```"CASH-OUT"```, ```"DEBIT"```, ```"PAYMENT"``` or ```"TRANSFER"```.

### amount
The amount of the transaction in local currency.

### nameOrig
The customer who started the transaction.

### oldbalanceOrg
The initial balance before the transaction.

### newbalanceOrg
The customer's balance after the transaction.

### nameDest
The recipient ID of the transaction.

### oldbalanceDest
The initial recipient balance before the transaction.

### newbalanceDest
The recipient's balance after the transaction.

### isFraud
Identifies a fraudulent transaction (1) and non fraudulent (0)

### isFlaggedFraud
Flags illegal attempts to transfer more than 200.000 in a single transaction.

## What is the XGBoost model?

XGBoost stands for e**X**treme **G**radient **Boost**. It's a powerful machine learning algorithm used for classification tasks.

It's a gradient-boosted decision tree model

For more details, here's the [Nvidia glossary page for XGBoost](https://www.nvidia.com/en-us/glossary/data-science/xgboost/#:~:text=XGBoost%2C%20which%20stands%20for%20Extreme,%2C%20classification%2C%20and%20ranking%20problems.). 

## Parameters of the XGBoost model

## Bayesian Hyperparameter Tuning

## Why I use XGBoost and Bayesian Hyperparameter tuning

## Code

In [ ]:
# import all necessary libraries
from sys import modules
from os import listdir
import pandas as pd
import numpy as np
if not "seaborn" in modules:
    %pip install seaborn
import seaborn as sns
if not "matplotlib" in modules:
    %pip install matplotlib
import matplotlib as plt
if not "xgboost" in modules:
    %pip install xgboost
from xgboost import XGBClassifier
if not "sklearn" in modules:
    %pip install sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
if not "kaggle" in modules:
    %pip install kaggle
import kaggle
if not "hyperopt" in modules:
    %pip install hyperopt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
# random state to be used for the whole program
random_state = 3

In [58]:
space = {
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 1,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': 180,
    'seed': 0
    }

In [59]:
def objective(space):
    clf = XGBClassifier(
        n_estimators=space['n_estimators'],
        max_depth=int(space['max_depth']),
        gamma=space['gamma'],
        reg_alpha=int(space['reg_alpha']),
        min_child_weight=int(space['min_child_weight']),
        colsample_bytree=int(space['colsample_bytree'])
    )
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    clf.fit(
        X_train,
        y_train,
        eval_set=evaluation,
        eval_metric="auc",
        early_stopping_rounds=10,
        verbose=False
    )
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

In [3]:
# set up kaggle API access
# make sure to upload your API key, which should be in a file called "kaggle.json"
if "kaggle.json" in listdir():
    if not ".kaggle" in listdir():
        %mkdir ~/.kaggle/
    if not ".kaggle" in listdir(".kaggle"):
        %mv kaggle.json ~/.kaggle/

# copy this into the terminal
# chmod 600 ~/.kaggle/kaggle.json

You can download the dataset [here](https://www.kaggle.com/code/arjunjoshua/predicting-fraud-in-financial-payment-services/input).

In [4]:
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")

In [60]:
df.shape
# (6362620, 11)

(6362620, 11)

In [6]:
# see if anything pops out with basic stats
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [7]:
# look at data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [8]:
df.columns
# step
# Maps a unit of time in the real world. In this case 1 step is 1 hour of time.

# type
# CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER

# amount
# amount of the transaction in local currency

# nameOrig
# customer who started the transaction

# oldbalanceOrg
# initial balance before the transaction

# newbalanceOrg
# customer's balance after the transaction.

# nameDest
# recipient ID of the transaction.

# oldbalanceDest
# initial recipient balance before the transaction.

# newbalanceDest
# recipient's balance after the transaction.

# isFraud
# identifies a fraudulent transaction (1) and non fraudulent (0)

# isFlaggedFraud
# flags illegal attempts to transfer more than 200.000 in a single transaction.

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [9]:
# find out how much fraud occurs
n_frauds = df[df["isFraud"] == 1].shape[0]
print(f'{n_frauds} frauds out of {df.shape[0]}')
print(f"{n_frauds/df.shape[0]} %")
# 8213 frauds out of 6362620
# 0.001290820448180152 %

8213 frauds out of 6362620
0.001290820448180152 %


In [10]:
# find out the success rate for flagging transactions as fraud
mask1 = (df["isFlaggedFraud"] == 1) & (df["isFraud"] == 1)
correct_fraud_flags = df[mask1].shape[0]
mask2 = (df["isFlaggedFraud"] == 0) & (df["isFraud"] == 1)
incorrect_fraud_flags = df[mask2].shape[0]
print(f"                    frauds: {n_frauds}")
print(f"  frauds flagged correctly: {correct_fraud_flags}")
print(f"frauds flagged incorrectly: {incorrect_fraud_flags}")
fraud_flag_success_rate = 100 * correct_fraud_flags / n_frauds
print(f"   fraud flag success rate: {round(fraud_flag_success_rate, 3)}%")

                    frauds: 8213
  frauds flagged correctly: 16
frauds flagged incorrectly: 8197
   fraud flag success rate: 0.195%


In [11]:
# see if any columns should be converted to numerical values
for column in df.columns:
    print(column, df.dtypes[column], df[column].unique()[:5])

step int64 [1 2 3 4 5]
type object ['PAYMENT' 'TRANSFER' 'CASH_OUT' 'DEBIT' 'CASH_IN']
amount float64 [ 9839.64  1864.28   181.   11668.14  7817.71]
nameOrig object ['C1231006815' 'C1666544295' 'C1305486145' 'C840083671' 'C2048537720']
oldbalanceOrg float64 [170136.  21249.    181.  41554.  53860.]
newbalanceOrig float64 [160296.36  19384.72      0.    29885.86  46042.29]
nameDest object ['M1979787155' 'M2044282225' 'C553264065' 'C38997010' 'M1230701703']
oldbalanceDest float64 [    0. 21182. 41898. 10845.  5083.]
newbalanceDest float64 [     0.    40348.79 157982.12  51513.44  16896.7 ]
isFraud int64 [0 1]
isFlaggedFraud int64 [0 1]


In [12]:
# see if anything pops out correlation-wise
corr = df.corr()
# corr
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# );

/var/folders/yn/f99zsf6d0_lc4tmwbl4zjwg80000gn/T/ipykernel_811/1216102543.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


In [13]:
X = df[df.columns.drop(["isFraud", "isFlaggedFraud"])].select_dtypes(
    include=["int64", "float64"]
)
y = df["isFraud"]

In [14]:
X.sample(5)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
3846334,282,78501.41,11021.00,0.00,387425.31,465926.72
2944073,230,56614.18,38813.00,0.00,3508031.07,3564645.25
5784444,400,40480.76,0.00,0.00,66636.52,107117.28
4608418,329,12143.62,20561.04,8417.43,0.00,0.00
47499,9,35311.34,0.00,0.00,0.00,0.00


In [30]:
# use xg boost
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=.25,
    random_state=random_state
)

In [35]:
parameter_grid_rscv = {
    "reg_lambda": [0.01],
    "reg_alpha": [0.1],
    "random_state": [3],
    "n_jobs": [-1],
    "n_estimators": [175, 200, 225],
    "learning_rate": [0.1, 0.08, 0.12],
    "booster": ["dart"],
}   

In [36]:
xgb = XGBClassifier()

In [37]:
randomized_search_CV = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=parameter_grid_rscv,
    random_state=random_state,
    cv=3,
    n_iter=10,
    n_jobs=-1
)

In [ ]:
# run 1
# 151 minutes
# run 2
# 578 minutes
# run 3
# ___ minutes
rscv = randomized_search_CV.fit(
    X=X_train,
    y=y_train
)

In [ ]:
#  run 1
# {'reg_lambda': 0.01,
# 'reg_alpha': 0.1,
# 'random_state': 3,
# 'n_jobs': -1,
# 'n_estimators': 150,
# 'learning_rate': 0.1,
# 'booster': 'dart'}
#  run 2
# {
#     "reg_lambda": 0.01,
#     "reg_alpha": 0.1,
#     "random_state": 3,
#     "n_jobs": -1,
#     "n_estimators": 175,
#     "learning_rate": 0.1,
#     "booster": dart,
# }
# run 3  
result = "{\n"
for k in rscv.best_params_:
    result += f"    \"{k}\": {rscv.best_params_[k]},\n"
result += "}"
print(result)


In [56]:
print(f"    rscv.best_score_={rscv.best_score_}")
# print(f"rscv.best_estimator_={rscv.best_estimator_}")
# print(f"    rscv.best_index_={rscv.best_index_}")
# print(f"    rscv.cv_results_={rscv.cv_results_}")

# 1
# rscv.best_score_=0.9996938368156515

    rscv.best_score_=0.9996938368156515


That's way too high. I assume it's overfitted. Let's work on that. Here are some parameters to adjust:

```colsample_bytree``` The ratio of features used. Lower ratios avoid over-fitting.

```subsample``` The ratio of the training instances used. Lower ratios avoid over-fitting.

```max_depth``` The maximum depth of a tree. Lower values avoid over-fitting.

```gamma``` The minimum loss reduction required to make a further split. Larger values avoid over-fitting.

```eta``` The learning rate of our GBM (i.e. how much we update our prediction with each successive tree). Lower values avoid over-fitting.

```min_child_weight``` The minimum sum of instance weight needed in a leaf, in certain applications this relates directly to the minimum number of instances needed in a node. Larger values avoid over-fitting.


In [46]:
# XGBoost (and other gradient boosting machine routines too) has a number of parameters that can be tuned to avoid over-fitting. I will mention some of the most obvious ones. For example we can change:

# the ratio of features used (i.e. columns used); colsample_bytree. Lower ratios avoid over-fitting.

# the ratio of the training instances used (i.e. rows used); subsample. Lower ratios avoid over-fitting.

# the maximum depth of a tree; max_depth. Lower values avoid over-fitting.

# the minimum loss reduction required to make a further split; gamma. Larger values avoid over-fitting.

# the learning rate of our GBM (i.e. how much we update our prediction with each successive tree); eta.
# Lower values avoid over-fitting.

# the minimum sum of instance weight needed in a leaf, in certain applications this relates directly to
# the minimum number of instances needed in a node; min_child_weight. Larger values avoid over-fitting.

xgb_rscv = XGBClassifier(
    n_estimators=175,
    learning_rate=0.1,
    booster="dart",
    reg_alpha=0.1,
    reg_lambda=0.01,
    n_jobs=-1,
    random_state=random_state
)

In [40]:
# parameter_grid_gscv = {
#     "n_estimators": [125, 150, 175],
#     "learning_rate": [0.1, 0.12, 0.08],
#     "booster": ["gbtree", "gblinear", "dart"],
#     "n_jobs": [-1],
#     "reg_alpha": [0.1, 0.08, 0.12],
#     "reg_lambda": [0.012, 0.008, 0.01],
#     "random_state": [random_state]
# }

In [42]:
# grid_search_cv = GridSearchCV(
#     estimator=xgb,
#     param_grid=parameter_grid_gscv,
#     cv=3,
#     n_jobs=-1
# )

In [ ]:
# gscv = grid_search_cv.fit(
#     X=X_train,
#     y=y_train
# )

In [ ]:
# result = "{\n"
# for k in gscv.best_params_:
#     result += f"    \"{k}\": {rscv.best_params_[k]},\n"
# result += "}"
# print(result)

In [48]:
# 35 minutes
xgb_rscv.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=175, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=3, ...)

In [49]:
# predict fraud with XGBoost model
y_predictions = xgb_rscv.predict(X_test)

In [52]:
print(type(y_predictions))
print(len(y_predictions))
print(y_predictions)

<class 'numpy.ndarray'>
1590655
[0 0 0 ... 0 0 0]
